In [1]:
import pandas as pd
import numpy as np
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import glob
import os
import ast
import seaborn as sns

from scipy.spatial import distance

from konlpy.tag import Okt
okt=Okt()

import pickle

In [2]:
city_names = ['강남','강동','강북','강서','관악','광진','구로','금천','노원','도봉','동대문','동작','마포','서대문','서초','성동','성북','송파','양천','영등포','용산','은평','종로','중구','중랑','강서','금정','기장','남구','동구','동래','부산진','북구','사상','사하','서구','수영','연제','영도','중구','해운대',
                       '광산','남구','동구','북구','서구','강화','계양','남동','동구','미추홀','부평','서구','연수','옹진','중구',
                       '대덕','동구','서구','유성','중구',
                       '가평','고양','과천','광명','광주','구리','군포','김포','남양주','동두천','부천','성남','수원','시흥','안산','안성','안양','양주','양평','여주','연천','오산','용인','의왕','의정부','이천','파주','평택','포천','하남','화성',
                       '남구','달서','달성','동구','북구','서구','수성','중구',
                       '남구','동구','북구','울주','중구',
                       '세종','강릉','고성','동해','삼척','속초','양구','양양','영월','원주','인제','정선','철원','춘천','태백','평창','홍천','화천','횡성',
                       '괴산','단양','보은','영동','옥천','음성','제천','증평','진천','청주','충주',
                       '계룡','공주','금산','논산','당진','보령','부여','서산','서천','아산','예산','천안','청양','태안','홍성',
                       '고창','군산','김제','남원','무주','부안','순창','완주','익산','임실','장수','전주','정읍','진안',
                       '강진','고흥','곡성','광양','구례','나주','담양','목포','무안','보성','순천','신안','여수','영광','영암','완도','장성','장흥','진도','함평','해남','화순',
                       '경산','경주','고령','구미','군위','김천','문경','봉화','상주','성주','안동','영덕','영양','영주','영천','예천','울릉','울진','의성','청도','청송','칠곡','포항',
                       '거제','거창','고성','김해','남해','밀양','사천','산청','양산','의령','진주','창녕','창원','통영','하동','함안','함양','합천',
                       '서귀포','제주',
                       '오사카','도쿄','후쿠오카','삿포로','오키나와','교토','나고야','유후','고베','요코하마',
                       '상하이','톈진','베이징','칭다오','장자제','광저우','하얼빈','하이난','샤먼','시안',
                       '다낭','냐짱','호찌민','푸꾸옥','하노이','달랏','호이안',
                       '방콕','푸껫','치앙마이','파타야','코사무이','크라비','후아힌',
                       '세부','보라카이','보홀','마닐라','팔라완',
                       '홍콩','타이페이','가오슝','타이중','타이난','화롄',
                       '마카오','싱가포르','코타키나발루','쿠알라룸푸르',
                       '발리','자카르타','롬복','빈탄섬','프라하',"바르셀로나", "마요르카", "마드리드", "세비야", "그라나다",
                       "프랑크푸르트", "뮌헨", "베를린", "함부르크", "드레스덴", "쾰른", "하이델베르크",
                       "빈", "잘츠부르크", "할슈타트",'블라디보스토크','모스크바','상트페테르부르크',
                       "런던", "에든버러","본머스","글래스고","리버풀","맨체스터","옥스퍼드",'암스테르담','헤이그','로테르담',
                       '파리','니스','마르세유','스트라스부르','보르도','몽생미셸','콜마르','리옹','아를','아비뇽',
                       '뉴욕','LA','샌프란시스코','라스베이거스','시카고','시애틀','워싱턴','보스턴','샌디에이고','괌','오하우','빅아일랜드',
                       '로마','피렌체','밀라노','베네치아','나폴리','시칠리아',
                       '시드니','멜버른','브리즈번','골드코스트','퍼스','캔버라','케언즈','애들레이드','율라라',
                       '케이프타운','요하네스버그','밴쿠버','토론토','오타와','퀘벡','몬트리올','나이아가라폭포시티',
                       '인터라켄','취리히','루체른','그린델발트','베른','체르마트','제네바','몽트뢰','라우터브루넨',
                       '부다페스트']

country_names = ['일본','중국','베트남','태국','필리핀','홍콩','대만','마카오','싱가포르','말레이시아',
                 '인도네시아','체코','스페인','독일','오스트리아','러시아','영국','네덜란드','프랑스',
                 '미국','이탈리아','호주','남아프리카공화국','캐나다','스위스','헝가리','서울',
                 '부산','광주','인천','대전','경기','대구','울산','세종','강원',
                 '충북','충남','전북','전남','경북','경남','제주']

In [3]:
# Load the DataFrame from the pickle file
with open('okt_불용어제거_해외국내_토큰.pkl', 'rb') as f:
    loaded3_df = pickle.load(f)

In [4]:
#해외 데이터만 따로
overseas = loaded3_df[0:60349]
overseas

,내용,country,city
0,"[도시, 유영, 최근, 미국여행, 개별, 가기, 트래블, 진행, 서부, 세미, 패키...",미국,LA
1,"[요즘, 핫, 핫플, 분위기, 분, 주목, 맛집, 리퍼블릭, 카페, 캐, 주얼, 베...",미국,LA
2,"[엘에이, 한인, 택시, 헬로, 카샤, 반년, 지난, 캘리포니아, 히히히, 캘리, ...",미국,LA
3,"[이제, 한파, 누, 며칠, 날씨, 썩, 요즘, 날, 고해, 주로, 실내, 쇼핑몰,...",미국,LA
4,"[서부, 그랜드캐년, 후기, 날씨, 서부, 그랜드캐년, 후기, 날씨, 서부, 그랜드...",미국,LA
...,...,...,...
60344,"[야쿠, 니, 키치, 함바, 오믈렛, 오므라이스, 맛집, 야쿠인, 인기, 양식, 집...",일본,후쿠오카
60345,"[초이, 남편, 컨셉, 먹방, 걸, 인생, 낙, 부부, 무조건, 음식, 공유, 먹방...",일본,후쿠오카
60346,"[데, 하카타, 매우, 요소, 중, 목욕탕, 도미, 인, 체인, 경우, 인기, 몸,...",일본,후쿠오카
60347,"[갑자기, 결정, 출장, 즐거움, 매일, 저녁, 로컬, 맛집, 기웃, 거리, 대도시...",일본,후쿠오카


In [5]:
def clean_punc(text, punct, mapping, unique): 
    for p in mapping: 
        text = text.replace(p, "") 
    for p in punct: 
        text = text.replace(p, "")
    for p in unique:
        text = text.replace(p, "")
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials: 
        text = text.replace(s, "") 
    return text.strip()

In [6]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'+'↑↓▼▲▶◆♥·'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi' }

remove_word = np.array(["여행", "계획", "자유여행", "사람","톨레도","예약","이동",
                       "공항","사진","일정","숙소","투어","추천","방문",
                       "생각","때문","도착","이용","정도","코스","주문","페이지",
                        "유럽","다시","구경","호이안","괴테","그냥","모습","한국",
                       "블로그", "오늘", "공지", "위치", "보고", "재생", "목록",
                        "버스",'시작', '처음', '준비', '댓글', '타고', '느낌','주차장',
                       '발트', '융프라우','나트랑','그린','스코틀랜드','알함브라',
                       '블로거','공감','패스', '라스베가스', '기차','엄마',
                       '후기', '보기', '메모', '전체' ,'화면', '작성', '비행기',
                        '택시', '출발', '미리','가격','사용','방법','확인','경우',
                        '체크','티켓','구매','인쇄','카테고리','최상','이전','직원',
                       '정보','포함','이제','사실', '포스팅', '이유','선택','한번','지금','친구',
                       '세계', '제목', '지역', '날씨', '관광','위해','항공권', "하와이",
                        "마지막", "카드", "탑승",'불가','지하철', '항공',
                       "도시","패키지","여행지",'진행'])
#remove_word는 불용어는 아닌데 토픽으로 잡히면 해석하기 어려운 단어
#도시, 나라 이름 또한 토픽으로 잡힐 확률이 높아서 삭제
unique = city_names
unique = list(np.append(unique, country_names))
unique = list(np.append(unique, remove_word))
unique = {string : "" for string in unique}

In [7]:
filtered_text = []

for sentence in overseas['내용']:
    filtered_sentence = [word for word in sentence if word not in unique]
    filtered_text.append(filtered_sentence)

In [8]:
overseas["내용"] = filtered_text
over_tokenized_doc = overseas['내용']

C:\Temp\ipykernel_23892\3767725203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overseas["내용"] = filtered_text


In [9]:
# 역토큰화 (토큰화 작업을 되돌림)
over_detokenized_doc = []
for i in range(len(overseas)):
    t = ' '.join(over_tokenized_doc[i])
    over_detokenized_doc.append(t)


In [13]:
### max_feature는 tf-idf vector의 최대 feature를 설정해주는 파라미터
vectorizer = TfidfVectorizer(max_features=150)
X_over = vectorizer.fit_transform(over_detokenized_doc)

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X_over.shape)

TF-IDF 행렬의 크기 : (60349, 150)


In [14]:
#LDA 토픽 개수는 혼잡도 등을 고려하여 결정

lda_model = LatentDirichletAllocation(n_components=6,learning_method='batch',random_state=777,max_iter=30)

In [15]:
lda_top_over = lda_model.fit_transform(X_over)

In [16]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=7):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('리조트', 1916.88), ('비치', 1443.76), ('수영장', 1140.5), ('바다', 1084.89), ('해변', 1044.73), ('마사지', 1036.08), ('객실', 1022.48)]
Topic 2: [('마켓', 1102.9), ('야시장', 901.02), ('쇼핑', 876.39), ('시장', 802.3), ('거리', 547.36), ('구입', 543.52), ('기념품', 426.42)]
Topic 3: [('맛집', 2001.77), ('메뉴', 1599.96), ('음식', 1459.03), ('커피', 1423.92), ('카페', 1350.51), ('맥주', 1239.94), ('식당', 1151.78)]
Topic 4: [('마을', 1396.28), ('호수', 1291.96), ('풍경', 1198.49), ('공원', 1002.1), ('열차', 968.5), ('전망대', 743.99), ('겨울', 634.97)]
Topic 5: [('성당', 1673.96), ('광장', 1611.68), ('대성당', 1247.46), ('궁전', 1000.67), ('야경', 965.37), ('건물', 867.74), ('다리', 802.93)]
Topic 6: [('박물관', 1369.41), ('가이드', 1189.33), ('미술관', 1007.41), ('작품', 908.6), ('파크', 726.15), ('역사', 723.24), ('공연', 722.78)]


C:\Users\145kj\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
overseas_data = pd.concat([overseas[["country","city"]], pd.DataFrame(lda_top_over)],axis=1)
overseas_data

,country,city,0,1,2,3,4,5
0,미국,LA,0.030883,0.845057,0.030955,0.031027,0.031057,0.031021
1,미국,LA,0.026171,0.026203,0.868926,0.026232,0.026216,0.026252
2,미국,LA,0.057054,0.056815,0.056709,0.056709,0.056527,0.716186
3,미국,LA,0.027198,0.027023,0.026910,0.864864,0.026938,0.027068
4,미국,LA,0.029567,0.029682,0.029565,0.851965,0.029555,0.029666
...,...,...,...,...,...,...,...,...
60344,일본,후쿠오카,0.040220,0.040202,0.799062,0.040222,0.040098,0.040196
60345,일본,후쿠오카,0.056689,0.056351,0.717739,0.056385,0.056164,0.056673
60346,일본,후쿠오카,0.818600,0.036331,0.036432,0.036184,0.036147,0.036306
60347,일본,후쿠오카,0.034745,0.034743,0.826393,0.034703,0.034678,0.034737


In [20]:
overseas_city_topic = overseas_data.groupby(["country","city"]).mean().reset_index().copy()
overseas_city_topic

,country,city,0,1,2,3,4,5
0,남아프리카공화국,요하네스버그,0.099022,0.099754,0.313179,0.128189,0.053696,0.306159
1,남아프리카공화국,케이프타운,0.154944,0.067494,0.328794,0.221184,0.051321,0.176264
2,네덜란드,로테르담,0.040427,0.179278,0.242250,0.154351,0.147905,0.235789
3,네덜란드,암스테르담,0.043662,0.102179,0.244954,0.127401,0.140370,0.341435
4,네덜란드,헤이그,0.063047,0.071375,0.184957,0.158338,0.095558,0.426724
...,...,...,...,...,...,...,...,...
130,호주,율라라,0.502015,0.037420,0.128391,0.172402,0.075690,0.084082
131,호주,캔버라,0.070856,0.065541,0.269818,0.255466,0.044511,0.293808
132,호주,케언즈,0.225715,0.097799,0.192507,0.260180,0.034170,0.189630
133,호주,퍼스,0.146781,0.115683,0.273998,0.267310,0.046639,0.149588


In [21]:
overseas_city_topic.to_csv("LDA해외비율.csv", encoding="cp949", index=False)